In [ ]:
!pip install -U torch==1.10.0 torchtext==0.11.0

# Reload environment
exit()

     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x21fe000 @  0x7f4298ca8615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 18 kB/s 
     |████████████████████████████████| 8.0 MB 40.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

In [ ]:
import random
import re
import math
import pandas as pd
import spacy
import torch
import torch.optim as optim
import torch.nn as nn
from torch import Tensor
from torchtext.legacy import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
g_path = "/content/drive/My Drive/"
data_path = g_path + "ml-data/"
code_path = g_path + "pytorch-nlp/"

Mounted at /content/drive


In [ ]:
data_fl = 'data/IMDB_review_sentiment_small.csv'

In [ ]:
#reproducing the same result
SEED = 2021
torch.manual_seed(SEED)

In [ ]:
spacy_en = spacy.load('en')
def clean_data(texts):
    cleaned_text = []
    for text in texts:
        # remove break
        text = text.replace('br', '')
        # remove punctuation
        text = re.sub('[^a-zA-Z0-9]', ' ', text)
        # remove multiple spaces
        text = re.sub(r' +', ' ', text)
        # remove newline
        text = re.sub(r'\n', ' ', text)
        # strip the text
        text = text.strip()
        # lower the text
        text = text.lower()

        if text != '':
          cleaned_text.append(text)
    return cleaned_text

def tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

TEXT = data.Field(preprocessing=clean_data,tokenize=tokenizer,batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)
fields = [('text',TEXT),('label', LABEL)]

In [ ]:
#loading the entire data
def load_data():
  imdb_data = data.TabularDataset(path = data_path+data_fl,format = 'csv', fields = fields, skip_header = True)
  return imdb_data

imdb_data = load_data() 
print(vars(imdb_data.examples[0]))
print(imdb_data.examples[0].text, imdb_data.examples[0].label)

{'text': ['one', 'of', 'the', 'other', 'reviewers', 'has', 'mentioned', 'that', 'after', 'watching', 'just', '1', 'oz', 'episode', 'you', 'll', 'be', 'hooked', 'they', 'are', 'right', 'as', 'this', 'is', 'exactly', 'what', 'happened', 'with', 'me', 'the', 'first', 'thing', 'that', 'struck', 'me', 'about', 'oz', 'was', 'its', 'utality', 'and', 'unflinching', 'scenes', 'of', 'violence', 'which', 'set', 'in', 'right', 'from', 'the', 'word', 'go', 'trust', 'me', 'this', 'is', 'not', 'a', 'show', 'for', 'the', 'faint', 'hearted', 'or', 'timid', 'this', 'show', 'pulls', 'no', 'punches', 'with', 'regards', 'to', 'drugs', 'sex', 'or', 'violence', 'its', 'is', 'hardcore', 'in', 'the', 'classic', 'use', 'of', 'the', 'word', 'it', 'is', 'called', 'oz', 'as', 'that', 'is', 'the', 'nickname', 'given', 'to', 'the', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'it', 'focuses', 'mainly', 'on', 'emerald', 'city', 'an', 'experimental', 'section', 'of', 'the', 'prison', 'where', 'all', 'the',

In [ ]:
#splitting the data into training and validation dataset
def split_data(imdb_data):
  train_data, valid_data = imdb_data.split(split_ratio=0.7, random_state = random.seed(SEED))
  return train_data, valid_data

train_data, valid_data = split_data(imdb_data)

In [ ]:
#generate vocabulary
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))
#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

print("unk:- ", TEXT.vocab.stoi["<unk>"])
print("pad:- ", TEXT.vocab.stoi["<pad>"])
print("sos:- ", TEXT.vocab.stoi["<sos>"]) #not present in dictionary
print("The first word in vocab is ", TEXT.vocab.itos[0])
print("The second word in vocab is ", TEXT.vocab.itos[1])
print("The third word in vocab is ", TEXT.vocab.itos[2])
print("The last word in vocab is ", TEXT.vocab.itos[len(TEXT.vocab)-1])

.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
100%|█████████▉| 399999/400000 [00:17<00:00, 23516.79it/s]


Size of TEXT vocabulary: 466
Size of LABEL vocabulary: 2
unk:-  0
pad:-  1
sos:-  0
The first word in vocab is  <unk>
The second word in vocab is  <pad>
The third word in vocab is  the
The last word in vocab is  yes


In [ ]:
#preparing batches for training the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 5

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort=False, # Sort all examples in data using `sort_key`.
    sort_key = lambda x: len(x.text),
    sort_within_batch=True, # Use `sort_key` to sort examples in each batch.
    device = device)

In [ ]:
class SelfAttention(nn.Module):

    def __init__(self, conf):
        super(SelfAttention, self).__init__()

        self.embed_dim = conf["embed_dim"]
        self.heads = conf["heads"] #number of heads
        self.head_dim = conf["embed_dim"] // conf["heads"]

        assert (
            self.head_dim * self.heads == conf["embed_dim"]
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)

        self.fc_out = nn.Linear(self.heads * self.head_dim,  self.embed_dim)

    def forward(self, values, keys, query, mask):
        
        '''
        values ~ # [batch, seq_len, embed_dim]
        keys ~ # [batch, seq_len, embed_dim]
        query ~ # [batch, seq_len, embed_dim]
        mask ~ # [batch, 1, 1, seq_len]
        '''

        # Get number of training examples
        batch = query.shape[0]

        '''
        value_len ~ the max seq_len in values 
        key_len ~ the max seq_len in keys 
        query_len ~ the max seq_len in query 
        here all of them are same
        '''
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        # [batch, seq_len, embed_dim] --converted--> [batch, seq_len, heads, head_dim]
        # [batch, value_len, heads, head_dim]
        values = values.reshape(batch, value_len, self.heads, self.head_dim)

        # [batch, seq_len, embed_dim] --converted--> [batch, seq_len, heads, head_dim]
        # [batch, key_len, heads, head_dim]
        keys = keys.reshape(batch, key_len, self.heads, self.head_dim)

        # [batch, seq_len, embed_dim] --converted--> [batch, seq_len, heads, head_dim]
        # [batch, query_len, heads, head_dim]
        query = query.reshape(batch, query_len, self.heads, self.head_dim)



        # [batch, value_len, heads, head_dim]
        values = self.values(values  # [batch, value_len, heads, head_dim]
                             )
        # [batch, key_len, heads, head_dim]
        keys = self.keys(keys  # [batch, key_len, heads, head_dim]
                         )
        # [batch, query_len, heads, head_dim]
        queries = self.queries(query  # [batch, query_len, heads, head_dim]
                               )

        
        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm
        # [batch, head, query_len, heads]
        energy = torch.einsum("nqhd,nkhd->nhqk", 
                              [queries, # [batch, query_len, heads, head_dim]
                               keys # [batch, key_len, heads, head_dim]
                               ])

        # Mask padded indices so their weights become 0
        if mask is not None:
            # energy = energy.masked_fill(mask == 0, float("-1e20"))
            energy = energy.masked_fill(mask == 0, float("0"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability ~ sqrt(d_k)
        # attention shape: [batch, heads, query_len, key_len]
        attention = torch.softmax(energy / (self.embed_dim ** (1 / 2)), dim=3)

        '''
        here key_len and value_len are same ~ denoted by l 
        '''
        # [batch, query_len, heads, head_dim]
        out = torch.einsum("nhql,nlhd->nqhd", [attention,  # [batch, heads, query_len, key_len]
                                               values  # [batch, value_len, heads, head_dim]
                                               ])
        # [batch, query_len, heads, head_dim] --converted--> [batch, query_len, embed_dim]
        out = out.reshape(batch, query_len, self.heads * self.head_dim)


        # Linear layer doesn't modify the shape, final shape will be
        # [batch, query_len, embed_dim]
        out = self.fc_out(out # [batch, query_len, embed_dim]
                          )

        return out

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, conf):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(conf)
        self.norm1 = nn.LayerNorm(conf["embed_dim"])
        self.norm2 = nn.LayerNorm(conf["embed_dim"])

        self.feed_forward = nn.Sequential(
            nn.Linear(conf["embed_dim"], 
                      conf["forward_expansion"] * conf["embed_dim"]),
            nn.ReLU(),
            nn.Linear(conf["forward_expansion"] * conf["embed_dim"], 
                      conf["embed_dim"]),
        )

        self.dropout = nn.Dropout(conf["dropout"])

    def forward(self, value, key, query, mask):

      '''
      value ~ # [batch, seq_len, embed_dim]
      key ~ # [batch, seq_len, embed_dim]
      query ~ # [batch, seq_len, embed_dim]
      mask ~ # [batch, 1, 1, seq_len]
      '''

      '''
      Step 1: passing the value, key, and query to self attention layer
      '''
      # [batch, seq_len, embed_dim] 
      attention = self.attention(value,  # [batch, seq_len, embed_dim]
                                 key,  # [batch, seq_len, embed_dim]
                                 query,  # [batch, seq_len, embed_dim]
                                 mask  # [batch, 1, 1, seq_len]
                                 )

      '''
      Step 2: normalizing the output
      '''
      # [batch, seq_len, embed_dim]   
      # Add skip connection, run through normalization and finally dropout
      x = self.dropout(self.norm1(attention + query))
      forward = self.feed_forward(x)
      out = self.dropout(self.norm2(forward + x))

      return out

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, max_len: int = 5000, dropout: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def pos_mask(self, src, conf):
        src_mask = (src != conf["src_pad_idx"])
        # [batch, seq_len, embed_dim]
        # the values are True or False
        return src_mask

    def forward(self, x: Tensor, conf: dict) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        pos_mask_val = self.pos_mask(x, conf)

        x = x + self.pe[:x.size(0)]
        x = x.masked_fill(pos_mask_val == 0, float("0"))

        return self.dropout(x)

In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        conf
    ):

        super(Transformer, self).__init__()
        
        self.word_embedding = nn.Embedding(num_embeddings = src_vocab_size, 
                                           embedding_dim = conf["embed_dim"],
                                           padding_idx=1)
        
        # max_length is the max length of sentence in the entire input / batch
        max_length = 10 
        self.position_embedding = PositionalEncoding(conf["embed_dim"],
                                                      max_length)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(conf)
                for _ in range(conf["num_Tlayers"])
            ]
        )
        self.dropout = nn.Dropout(conf["dropout"])

    def forward(self, x, mask):

        '''
        x ~ # [batch, seq_len]
        mask ~ # [batch, 1, 1, seq_len]
        '''

        '''
        Step 1: pass through the embedding layer to convert text into vectors
        '''
        # x_embed ~ [batch, seq_len, embed_dim] 
        x_embed = self.word_embedding(x # [batch, seq_len]
                                      )  
        
        '''
        Step 2: position_embedding incorporates the position information
        '''
        if conf["ps_embed_enc_ind"]:
          x_embed_with_position = self.position_embedding(x_embed, conf)
          x_embed = x_embed_with_position

        out = self.dropout(x_embed)

        '''
        Step 3: passing the embeddings to the transformer block
        '''
        # In the Encoder the query, key, value are all the same
        for t_layer in self.layers:
          # [batch, seq_len, embed_dim]
          out = t_layer(out,  # [batch, seq_len, embed_dim]
                        out,  # [batch, seq_len, embed_dim]
                        out,  # [batch, seq_len, embed_dim]
                        mask  # [batch, 1, 1, seq_len]
                      )

        return out

In [ ]:
class TransformerClassifier(nn.Module):
    def __init__(
        self, src_vocab_size, conf
    ):

        super(TransformerClassifier, self).__init__()

        self.src_pad_idx = conf["src_pad_idx"]
        self.device = conf["device"]

        self.encoder = Transformer(
            src_vocab_size,
            conf
        )

        self.output_dim = 1
        #dense layer / linear layer
        self.fc = nn.Linear(conf["embed_dim"], self.output_dim)

        #activation function
        self.act = nn.Sigmoid()

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # [batch, 1, 1, seq_len]
        # the values are True or False
        return src_mask.to(self.device)

    def forward(self, src):
      
      '''
      src ~ # [batch, seq_len]
      '''

      '''
      Step 1: create mask for the attention layer
      '''
      # [batch, 1, 1, seq_len]
      # the values are True or False
      src_mask = self.make_src_mask(src # [batch, seq_len]
                                    )

      '''
      Step 2: passing the input to transformer
      '''
      # [batch, seq_len, embed_dim]
      enc_out = self.encoder(src,  # [batch, seq_len]
                             src_mask  # [batch, 1, 1, seq_len]
                             )
      '''
      Step 3: sum all the embed dim of each word in a sentence
      '''
      # [batch_size, embed_dim]
      weighted_out = enc_out.sum(dim=1)

      '''
      Step 4: feeding the weighted value to a linear layer
      '''
      # fc_out ~ [batch_size, output_dim]
      fc_out = self.fc(weighted_out  # [batch_size, embed_dim]
                       )

      '''
      Step 5: feeding the linear output to activation function
      '''
      # out ~ [batch_size, output_dim]
      out = self.act(fc_out)

      return out

In [ ]:
#define hyperparameters
vocab_size = len(TEXT.vocab)

conf = {
    "device" : torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "src_pad_idx" : 1,
    "embed_dim" : 6,
    "ps_embed_enc_ind": True,
    "num_Tlayers" : 2,
    "heads" : 2,
    "dropout" : 0.1,
    "forward_expansion" : 4
}

#instantiate the model
train_model = TransformerClassifier(vocab_size, conf)
train_model = train_model.to(device)

In [ ]:
#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def valid_model(valid_iterator, train_model, criterion):
  epoch_loss = 0
  epoch_acc = 0

  with torch.no_grad():
    for valid_batch in valid_iterator:
      
      #retrieve text and no. of words
      text, text_lengths = valid_batch.text
          
      #get prediction
      predictions = train_model(text)
      preds = predictions.squeeze(-1) #convert to 1D tensor

      #compute the loss
      loss = criterion(preds, valid_batch.label)

      #compute the binary accuracy
      acc = binary_accuracy(preds, valid_batch.label)

      # compute loss and accuracy
      epoch_loss += loss.item()
      epoch_acc += acc.item()

  valid_epoc_loss = epoch_loss / len(valid_iterator)
  valid_epoch_acc = epoch_acc / len(valid_iterator)

  return valid_epoc_loss, valid_epoch_acc

In [ ]:
#training the model

#define the optimizer
optimizer = optim.Adam(train_model.parameters())

#define the loss
criterion = nn.BCELoss()
criterion = criterion.to(device)

#set the model in training phase
train_model.train()

N_EPOCHS = 6
VALIDATION_EPOCH = 2

for epoch in range(N_EPOCHS+1):

  #initialize every epoch 
  epoch_loss = 0
  epoch_acc = 0

  for batch in train_iterator:
    #resets the gradients after every batch
    optimizer.zero_grad() 

    #retrieve text and no. of words
    text, text_lengths = batch.text

    #get prediction
    predictions = train_model(text)
    preds = predictions.squeeze(-1) #convert to 1D tensor

    #compute the loss
    loss = criterion(preds, batch.label)

    #compute the binary accuracy
    acc = binary_accuracy(preds, batch.label)   

    #backpropage the loss and compute the gradients
    loss.backward()

    #update the weights
    optimizer.step() 

    # compute loss and accuracy
    epoch_loss += loss.item()
    epoch_acc += acc.item()

  if epoch%VALIDATION_EPOCH == 0:
    train_model.eval() # set the model in eval phase
    valid_epoc_loss, valid_epoch_acc = valid_model(valid_iterator, train_model, criterion)
    train_model.train() # return back to training phase

    print("epoch:- ",epoch)
    print("training===> ","loss:- ", epoch_loss / len(train_iterator), "  accuracy:- ", epoch_acc / len(train_iterator))
    print("validation===> ","loss:- ", valid_epoc_loss, "  accuracy:- ", valid_epoch_acc)

  if epoch == N_EPOCHS-1:
    torch.save(train_model.state_dict(), code_path+"model/classification_model.pt")

epoch:-  0
training===>  loss:-  32.47466836656843   accuracy:-  0.37142858122076305
validation===>  loss:-  15.013405839602152   accuracy:-  0.6000000089406967
epoch:-  2
training===>  loss:-  32.84489893913269   accuracy:-  0.45714287034102846
validation===>  loss:-  16.63049602508545   accuracy:-  0.46666668355464935
epoch:-  4
training===>  loss:-  30.872309923171997   accuracy:-  0.42857143708637785
validation===>  loss:-  15.66829220453898   accuracy:-  0.533333346247673
epoch:-  6
training===>  loss:-  33.50921652998243   accuracy:-  0.3714285833495004
validation===>  loss:-  14.718185702959696   accuracy:-  0.5333333412806193


In [ ]:
###  Inference  ###

#define hyperparameters
vocab_size = len(TEXT.vocab)

conf = {
    "device" : torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "src_pad_idx" : 0,
    "embed_dim" : 6,
    "ps_embed_enc_ind": True,
    "num_Tlayers" : 2,
    "heads" : 2,
    "dropout" : 0.1,
    "forward_expansion" : 4
}

#instantiate the model
test_model = TransformerClassifier(vocab_size, conf)
test_model = test_model.to(device)

#loading the model
model_path = code_path+"model/classification_model.pt"
test_model.load_state_dict(torch.load(model_path))

test_model.eval() # set the model in eval phase


# test_sentence = "Are there any sports that you don't like?"
# test_sentence = "I love the movie"
# test_sentence = "I dislike the movie"
test_sentence = "I don't like the movie"

test_data = " ".join(clean_data(test_sentence.split(" "))) # clean the data
tokenized_test_data = tokenizer(test_data)  #tokenize the sentence

indexed_test_data = [TEXT.vocab.stoi[t] for t in tokenized_test_data]  #convert to integer sequence
txt_tensor = torch.LongTensor(indexed_test_data).to(device) #convert to tensor
txt_tensor_ip = txt_tensor.unsqueeze(1).T #reshape in form of batch,no. of words

length = [len(indexed_test_data)]  #compute no. of words
length_tensor_ip = torch.LongTensor(length) #convert to tensor 

prediction = test_model(txt_tensor_ip) #prediction

print(prediction.item())

0.4924543499946594
